In [36]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf
from keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM,
    Dense,
    Embedding,
    Bidirectional,
    Dropout,
    GlobalMaxPooling1D,
    Concatenate,
    BatchNormalization,
    Conv1D,
    ReLU,
    Input,
    GRU,
)
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.layers import Embedding, Flatten, Dense
import tensorflow_datasets as tfds
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, pos_tag, ne_chunk
import os
from keras.preprocessing.sequence import TimeseriesGenerator
from datetime import datetime
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import pipeline
from transformers import BertTokenizer, TFBertModel
import json
from nltk.tokenize import word_tokenize
from sklearn.calibration import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
test_df = pd.read_parquet("datasets/COQA-Conversation/test.parquet")
train_df = pd.read_parquet("datasets/COQA-Conversation/train_stanfordnlp.parquet")

In [29]:
def tokenize_text(text):
    return word_tokenize(text)


In [41]:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower()
                     not in stop_words]
    return " ".join(filtered_text)


def stem_text(text):
    stemmer = PorterStemmer()
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)


def extract_entities(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    ne_chunks = ne_chunk(pos_tags)

    entities = []
    for chunk in ne_chunks:
        if hasattr(chunk, "label") and chunk.label():
            if chunk.label() == "NE":
                entities.append(" ".join([c[0] for c in chunk]))
    return entities


def create_tfidf_vectorizer(df):
    vectorizer = TfidfVectorizer(max_features=10000, use_idf=True)
    # Fit and transform the text data in the DataFrame column
    tfidf_matrix = vectorizer.fit_transform(df["sentence"])
    # Convert the TF-IDF matrix to a DataFrame for visualization
    return tfidf_matrix.toarray()


def tokenize_sentences(df):
    # Flatten the DataFrame to iterate over all cells
    all_text = df.values.flatten()
    
    # Create a tokenizer
    tokenizer = Tokenizer(num_words=10000)
    
    # Fit tokenizer on all text
    tokenizer.fit_on_texts(all_text)
    
    # Tokenize all text
    sequences = tokenizer.texts_to_sequences(all_text)
    
    return sequences


def encode_emotions(emotions):
    encoder = LabelEncoder()
    return encoder.fit_transform(emotions)

def pad_sequences_with_zeros(X, maxlen):
    return pad_sequences(X, maxlen=maxlen)

In [61]:
test_df = test_df.map(remove_stopwords)

test_df = test_df.map(stem_text)

# train["entities"] = train["sentence"].apply(extract_entities)
# test["entities"] = test["sentence"].apply(extract_entities)
# val["entities"] = val["sentence"].apply(extract_entities)


test_tfidf = tokenize_sentences(test_df)

test_padded = pad_sequences_with_zeros(test_tfidf, 50)